# ITC - MPSI
---

# TP24  Détection de cycles dans un graphe orienté
Dans ce TP, on va adapter l'algorithme de parcours de graphe en profondeur pour détecter des cycles dans un graphe.

Je redonne ici le code pour le parcours en profondeur lorsque le graphe est représenté par une liste d'adjacence:

In [ ]:
from graphviz import Digraph
from IPython import display
from time import sleep
import random, string

#sac géré comme une pile
def sac_vide():
    return []

def ajoute_dans_sac(sac, e):
    sac.append(e)

def prends_dans_sac(sac):
    return sac.pop()

def est_vide(sac):
    return len(sac) == 0

def dessin_graphe(L):
    """L : représentation d'un graphe par liste d'adjacence
    renvoit un objet graphique représentant ce graphe"""
    graphe = Digraph(strict=True)  ## pour ne pas avoir d'arête multiple
    arcs = []
    for i in range(len(L)):
        graphe.node(str(i))
        arcs.extend([(str(i), str(j)) for j in L[i]])
    graphe.edges(arcs)
    return graphe

def colore_arc(s, t, couleur, gr, nom, pas_a_pas):
    """Colore l'arc s->t dans l'objet graphique gr d'identifiant nom"""
    gr.edge(str(s), str(t), color=couleur, penwidth='2')
    if pas_a_pas:        
        sleep(.5)
    display.update_display(gr, display_id=nom)

def colore_sommet(s, couleur, gr, nom, pas_a_pas):
    """Colore le sommet s dans l'objet graphique gr d'identifiant nom"""
    gr.node(str(s), style='filled', fillcolor=couleur)
    if pas_a_pas:        
        sleep(.5)
    display.update_display(gr, display_id=nom)  ## mise à jour du dessin
    
def parcours(ladj, s0, pas_a_pas=False):
    """Parccours en profondeur du graphe donné par la liste d'adjacence ladj, à partir du sommet s0
    pas_a_pas : si True l'affichage est plus lent"""
    gr = dessin_graphe(ladj)
    nom = ''.join(random.choice(string.ascii_letters) for x in range(15))  ## identifiant pour le graphe
    dis = display.display(gr, display_id=nom)   ## dessin du graphe initial
    decouverts = sac_vide()
    ajoute_dans_sac(decouverts, (s0, None))
    marques = [ False ] * len(ladj)
    while not est_vide(decouverts):
        s, p = prends_dans_sac(decouverts)
        if not marques[s]:
            marques[s] = True
            if p != None: ## si on sait d'où on vient pour s, on colore l'arc d'arrivée
                colore_arc(p, s,'#00A000', gr, nom, pas_a_pas)
                colore_sommet(s, 'grey', gr, nom, pas_a_pas) ## on marque le sommet pour le dessin
            else:
                colore_sommet(s, 'pink', gr, nom, pas_a_pas)  ## on modifie la couleur du sommet initial
            for t in ladj[s] :
                ajoute_dans_sac(decouverts, (t, s))
                colore_arc(s, t, 'red', gr, nom, pas_a_pas)

La fonction de parcours a été légèrement modifiée:
* en rouge les arcs qu'on ajoute dans le sac,
* en vert les arcs qu'on suit effectivement.

Il s'agit maintenant de détecter un cycle. Dans le cas orienté, il ne suffit pas de détecter deux façons distinctes d'atetindre un même sommet, car on pourrait se retrouver dans le cas du graphe suivant qui contient deux chemins du sommet 0 vers le somme 3, mais aucun cycle:

In [ ]:
parcours([[1,2],[3],[3],[]], 0, True)

Au contraire, dans le parcours du graphe suivant, à partir du sommet 3, on atteint le sommet 1 et il y a un cycle:

In [ ]:
parcours([[1],[2],[3],[1]], 0, True)

La différence entre les deux vient du fait que dans le premier cas, quand on considère l'arc $1\to 3$, on n'est pas dans un chemin qui est déjà passé par 3, alors que dans le deuxième cas, quand on considère l'arc $3\to 1$, on est dans un chemin qui est déjà passé par 1.

Pour tenir compte de cette remarque et mettre en place la détection de cycle à partir du parcours en profondeur, on a besoin de plus de renseignement que juste de savoir si le sommet a déjà été traité: on a besoin de savoir s'il a fini d'être traité (c'est-à-dire si on a explorés tous les chemines qui partent de lui) ou pas. Il faut donc avoir trois valeurs possibles pour les cases de la liste `marques`:
* `0` qui est la valeur initiale et signifie qu'on n'a encore jamais atteint le sommet,
* `1` qui est la valeur signifiant qu'on a commencé à traiter les chemins partant du sommet, mais qu'il en reste,
* `2` qui est la valeur signifiant qu'on a fini de traiter les chemins partant du sommet.

Ainsi, si on récupère une arête qui arrive dans un somme qui est en cours de traitement, c'est qu'on a une boucle.

Réécrivez le code pour détecter les cycles en partant d'un sommet donné (le cycle n'appartiendra pas nécessairement au cycle détecté, voir le deuxième exemple ci-dessus).

In [ ]:
#code

In [ ]:
#test1 : pas de cycle
ladj = [ [2*i+1, 2*i+2] for i in range (7) ]
ladj.extend([ [] for _ in range(8) ])
ladj[13] = [1]
parcours(ladj, 0, True)

In [ ]:
#test1 : un cycle
ladj = [ [2*i+1, 2*i+2] for i in range (7) ]
ladj.extend([ [] for _ in range(8) ])
ladj[13] = [1]
ladj[3].append(0)
parcours(ladj, 0, True)

On veut maintenant étendre à la recherche de cycle dans tout le graphe. Un fois l'algorithme déroulé à partir d'un sommet, on souhaite donc recommencer à partir d'un autre sommet si tous les sommets n'ont pas été atteints. Écrivez la fonction associée à cet algorithme.

## Une application
Le fichier [tarifs.txt](tarifs.txt) contient:
* une première ligne avec une liste de noms de villes séparés par des virgules (on considère dans la suite que chaque ville est indicée par sa place dans cette liste, en commençant par l'indice 0);
* sur les lignes suivantes, une liste de tarifs de péages présentée de la manière suivante sur chaque ligne (les valeurs étant séparées par des virgules):
  * ville de départ
  * ville d'arrivée
  * montant du péage
  
Certaines lignes contienne un montant de péage nul. On cherche à savoir s'il est possible de faire un circuit touristique sans passer par un péage et en retournant à son point de départ. (Bien sûr, si c'est le cas on voudrait connaître ce circuit.

À vous de jouer !